In [1]:
import sys
sys.path.insert(0, r"D:\LLMDelivery-LJ\SimWorld") 

import json
import os
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QLabel, QPushButton, QGraphicsRectItem
)
from PyQt5.QtGui import QPixmap, QPainter
from PyQt5.QtCore import Qt
import pyqtgraph as pg

# 世界尺度：如果你的 world 中点位/建筑坐标已经是 米×100（例如厘米），保持 100.0；
# 若以后路径(path)也写成×100，则把这里改成 1.0。
WORLD_SCALE = 100.0

class ProgenPOIVisualizer(QMainWindow):
    def __init__(self, map_path, building_def_path):
        super().__init__()
        self.map_path = map_path
        self.building_def_path = building_def_path
        self.setWindowTitle('Progen POI Map Viewer')

        # Load world & routes
        with open(map_path, 'r', encoding='utf-8') as f:
            self.data = json.load(f)
        self.nodes = self.data.get('nodes', [])
        self.bus_routes = self.data.get('bus_routes', [])

        # Load building defs
        with open(building_def_path, 'r', encoding='utf-8') as f:
            self.building_defs = self._load_building_defs(json.load(f))

        # GUI
        self.main_widget = QWidget()
        self.setCentralWidget(self.main_widget)
        layout = QVBoxLayout(self.main_widget)

        self.title_label = QLabel("Progen Map with POIs & Bus Routes")
        self.title_label.setAlignment(Qt.AlignCenter)
        layout.addWidget(self.title_label)

        self.plot_widget = pg.PlotWidget()
        layout.addWidget(self.plot_widget)

        self.save_button = QPushButton('Save Image')
        self.save_button.clicked.connect(self.save)
        layout.addWidget(self.save_button)

        # Plot style
        self.plot_widget.setBackground('#F0F8FF')
        self.plot_widget.showGrid(True, True, alpha=0.3)
        self.plot_widget.setAspectLocked(True)
        self.plot_widget.setMouseEnabled(x=True, y=True)
        self.plot_widget.setMenuEnabled(False)

        self.resize(1280, 720)
        self.draw_map()

    def _load_building_defs(self, defs_json):
        """
        期望结构：
        {
          "buildings": [
            { "type": "BP_Building_75_C", "bounds": {"width": 12.0, "height": 20.0} },
            ...
          ]
        }
        """
        result = {}
        for b in defs_json.get('buildings', []):
            btype = b.get('type')
            bounds = b.get('bounds', {}) or {}
            if btype and 'width' in bounds and 'height' in bounds:
                result[btype] = {
                    'width': float(bounds['width']),
                    'height': float(bounds['height']),
                }
        return result

    def draw_map(self):
        self.plot_widget.clear()

        counts = {
            'buildings': 0,
            'restaurants': 0,
            'stores': 0,
            'rest_areas': 0,
            'chargers': 0,
            'bus_stops': 0,
            'bus_routes': 0,
            'hospitals': 0,
            'car_rentals': 0
        }

        # ---- Draw buildings / POI-on-buildings ----
        for node in self.nodes:
            instance_name = node.get('instance_name', '')
            props = node.get('properties', {}) or {}
            loc = props.get('location', {}) or {}
            ori = props.get('orientation', {}) or {}

            # 现在点位已是 ×100 的世界坐标，这里直接用
            x = float(loc.get('x', 0.0))
            y = float(loc.get('y', 0.0))
            yaw = float(ori.get('yaw', 0.0))

            # 统一读取 poi_type，兼容旧数据的 type
            poi_type = props.get('poi_type') or props.get('type')

            # Buildings（包含被标注为 rest_area / hospital / car_rental 的建筑）
            if instance_name.startswith('BP_Building'):
                btype = instance_name  # 与 building_defs 的键一致（如 BP_Building_75_C）
                size = self.building_defs.get(btype, {'width': 1.0, 'height': 1.0})
                # 建筑尺寸同样放大到当前坐标系
                width = float(size['width']) * WORLD_SCALE
                height = float(size['height']) * WORLD_SCALE

                rect = QGraphicsRectItem(x - width / 2.0, y - height / 2.0, width, height)
                rect.setTransformOriginPoint(x, y)
                rect.setRotation(yaw)

                # 颜色按建筑上的 poi_type 区分
                if poi_type == 'restaurant':
                    color = '#E74C3C'      # 红：餐厅
                    counts['restaurants'] += 1
                elif poi_type == 'store':
                    color = '#3498DB'      # 蓝：商店
                    counts['stores'] += 1
                elif poi_type == 'rest_area':
                    color = '#9B59B6'      # 紫：休息区
                    counts['rest_areas'] += 1
                elif poi_type == 'hospital':
                    color = '#E91E63'      # 洋红：医院
                    counts['hospitals'] += 1
                elif poi_type == 'car_rental':
                    color = '#1ABC9C'      # 青绿：租车行
                    counts['car_rentals'] += 1
                else:
                    color = '#7F8C8D'      # 灰：普通建筑
                    counts['buildings'] += 1

                rect.setPen(pg.mkPen('k', width=1))
                rect.setBrush(pg.mkBrush(color))
                self.plot_widget.addItem(rect)

            # ---- Point POIs ----
            else:
                # 充电桩（点）
                if poi_type == 'charging_station' or instance_name.startswith('POI_ChargingStation'):
                    dot = pg.ScatterPlotItem(
                        pos=[(x, y)],
                        size=12,
                        pen=pg.mkPen('w'),
                        brush=pg.mkBrush('#27AE60'),  # 绿
                        symbol='o',
                        antialias=True
                    )
                    self.plot_widget.addItem(dot)
                    counts['chargers'] += 1

                # 公交站（点）
                elif poi_type == 'bus_station' or instance_name.startswith('POI_BusStation'):
                    stop = pg.ScatterPlotItem(
                        pos=[(x, y)],
                        size=14,
                        pen=pg.mkPen('#8E5A0A'),
                        brush=pg.mkBrush('#F39C12'),  # 橙
                        symbol='t',  # triangle
                        antialias=True
                    )
                    self.plot_widget.addItem(stop)
                    counts['bus_stops'] += 1

                # 兼容：若有点状的医院 / 租车行
                elif poi_type == 'hospital':
                    hosp = pg.ScatterPlotItem(
                        pos=[(x, y)],
                        size=14,
                        pen=pg.mkPen('#7D3C98'),
                        brush=pg.mkBrush('#BB8FCE'),
                        symbol='o',
                        antialias=True
                    )
                    self.plot_widget.addItem(hosp)
                    counts['hospitals'] += 1

                elif poi_type == 'car_rental':
                    car = pg.ScatterPlotItem(
                        pos=[(x, y)],
                        size=12,
                        pen=pg.mkPen('#1B4F72'),
                        brush=pg.mkBrush('#5DADE2'),
                        symbol='s',
                        antialias=True
                    )
                    self.plot_widget.addItem(car)
                    counts['car_rentals'] += 1

        # ---- Draw bus routes（path 乘 WORLD_SCALE，与站点对齐）----
        for route in self.bus_routes:
            path = route.get('path', [])
            if len(path) >= 2:
                xs = [float(p.get('x', 0.0)) * WORLD_SCALE for p in path]
                ys = [float(p.get('y', 0.0)) * WORLD_SCALE for p in path]
                line = pg.PlotDataItem(
                    x=xs,
                    y=ys,
                    pen=pg.mkPen('#FFA500', width=3.0),  # 橙线：公交线路
                    antialias=True
                )
                self.plot_widget.addItem(line)
                counts['bus_routes'] += 1

        print(
            f"✅ Drawing complete: "
            f"{counts['buildings']} buildings, "
            f"{counts['restaurants']} restaurants, "
            f"{counts['stores']} stores, "
            f"{counts['rest_areas']} rest areas, "
            f"{counts['chargers']} chargers, "
            f"{counts['bus_stops']} bus stops, "
            f"{counts['hospitals']} hospitals, "
            f"{counts['car_rentals']} car rentals, "
            f"{counts['bus_routes']} bus routes."
        )

    def save(self):
        out_path = os.path.join(os.path.dirname(self.map_path), 'progen_map_output_with_poi_bus.png')
        # 更稳妥的抓图方式
        pixmap = self.plot_widget.grab()
        if pixmap.save(out_path, 'PNG'):
            print(f'Image saved at {out_path}')
        else:
            print('❌ Failed to save image')


def main():
    import sys
    app = QApplication(sys.argv)

    # 使用包含 bus_routes 与新增 POIs 的 world 文件
    map_path = r"D:\LLMDelivery-LJ\Test_Data\test\progen_world_enriched.json"
    building_def_path = r"D:\LLMDelivery-LJ\Test_Data\test\buildings.json"

    viewer = ProgenPOIVisualizer(map_path, building_def_path)
    viewer.show()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()


✅ Drawing complete: 84 buildings, 5 restaurants, 5 stores, 2 rest areas, 15 chargers, 10 bus stops, 1 hospitals, 2 car rentals, 1 bus routes.


SystemExit: 0

c:\Users\Shadow\miniconda3\envs\citynav\lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
